<a href="https://colab.research.google.com/github/BushraAlmutiri/MedicalInsurance/blob/main/Copy_of_gp13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from flask import Flask, request, jsonify
import sqlite3
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error
import requests
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
app = Flask(__name__)

In [ ]:
df = pd.read_csv("insurance.csv") #loadDataSet

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info() #dataFrame

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(x='smoker', data=df)
plt.title('Smoker')
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
style.use('ggplot')
sns.countplot(x='sex', data=df)
plt.title('Gender')
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(x='region', data=df)
plt.title('Region')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(x='region', y='charges', data=df)
plt.title(' charges vs Region')

In [ ]:
plt.figure(figsize=(4,4))
sns.barplot(x='sex', y='charges',hue='smoker', data=df)
plt.title('Charges for smokers')

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='bmi', bins=20, kde=True)
plt.title("Distribution of BMI")
plt.xlabel("BMI")
plt.ylabel("Count")
plt.show()

In [ ]:
correlation_matrix = df.corr()
print("Correlation Matrix:")
print(correlation_matrix)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
correlation_matrix = df.corr(numeric_only=True)
plt.show()

In [ ]:
df['sex'] = df['sex'].apply({'male': 0, 'female': 1}.get)
df['smoker'] = df['smoker'].apply({'yes': 1, 'no': 0}.get)
df['region'] = df['region'].apply({'southwest': 1, 'southeast': 2, 'northwest': 3, 'northeast': 4}.get)

In [ ]:
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
y = df['charges']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
linear_regression_model = LinearRegression()
svr_model = SVR()
decision_tree_model = DecisionTreeRegressor()

In [ ]:
linear_regression_model.fit(X_train, y_train)

In [ ]:
svr_model.fit(X_train, y_train)

In [ ]:
decision_tree_model.fit(X_train, y_train)

In [ ]:
linear_regression_predictions = linear_regression_model.predict(X_test)

In [ ]:
svr_predictions = svr_model.predict(X_test)

In [ ]:
decision_tree_predictions = decision_tree_model.predict(X_test)

In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(64, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

In [ ]:
with open('linear_regression_model.pkl', 'wb') as f:
        pickle.dump(linear_regression_model, f)

with open('svr_model.pkl', 'wb') as f:
        pickle.dump(svr_model, f)

with open('decision_tree_model.pkl', 'wb') as f:
        pickle.dump(decision_tree_model, f)

In [ ]:
file_path = 'model.h5'
with open(file_path, 'w') as file:
    file.write('This is a sample file named model.')

In [ ]:
print(os.path.exists(file_path))

In [ ]:
model.save('model.h5')

In [ ]:
model = tf.keras.models.load_model('model.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
tfjs.converters.save_keras_model(model, './path/to/web/servable')

In [ ]:
data = {'age':19 , 'sex':1 , 'bmi':27.9 , 'children':0 , 'smoker':1 , 'region':1 }
index = [0]

In [ ]:
cust_df = pd.DataFrame(data, index)

In [ ]:
print(cust_df)

In [ ]:
cust_df = cust_df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]

In [ ]:
cost_pred_linear_regression = linear_regression_model.predict(cust_df)

In [ ]:
print("Linear Regression Prediction:", cost_pred_linear_regression)

In [ ]:
cost_pred_svr = svr_model.predict(cust_df)

In [ ]:
print("SVR Prediction:", cost_pred_svr)

In [ ]:
cost_pred_decision_tree = decision_tree_model.predict(cust_df)

In [ ]:
print("Decision Tree Prediction:", cost_pred_decision_tree)

In [ ]:
linear_regression_mse = mean_squared_error(y_test, linear_regression_predictions)

In [ ]:
print("Mean Squared Error Linear Regression:", linear_regression_mse)

In [ ]:
svr_mse = mean_squared_error(y_test, svr_predictions)

In [ ]:
print("Mean Squared Error SVR:", svr_mse)

In [ ]:
decision_tree_mse = mean_squared_error(y_test, decision_tree_predictions)

In [ ]:
print("Mean Squared Error Decision Tree :", decision_tree_mse)

In [ ]:
conn = sqlite3.connect('medical_insurance.db')

In [ ]:
c = conn.cursor()

In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS predictions
       (age INTEGER, sex INTEGER, bmi REAL, children INTEGER, smoker INTEGER, region INTEGER,
        linear_regression_prediction REAL, svr_prediction REAL, decision_tree_prediction REAL, best_model TEXT,  email TEXT, name TEXT)''')

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    input_data = request.json

In [ ]:
input_data = {
        'age': 25,
        'sex': 1,
        'bmi': 27.9,
        'children': 2,
        'smoker': 0,
        'region': 3,
    }

In [ ]:
if input_data:
        age = input_data.get('age')
        sex = input_data.get('sex')
        bmi = input_data.get('bmi')
        children = input_data.get('children')
        smoker = input_data.get('smoker')
        region = input_data.get('region')

In [ ]:
def check_input_data(age, sex, bmi, children, smoker, region):
    if None in (age, sex, bmi, children, smoker, region):
        return jsonify({'error': 'Some keys are missing in the input data.'}), 400

In [ ]:
cust_df = pd.DataFrame({'age': [age], 'sex': [sex], 'bmi': [bmi], 'children': [children], 'smoker': [smoker], 'region': [region]})

In [ ]:
input_data = np.array([[25, 1, 27.9, 2, 0, 3]])  # Convert input data to a 2D array
linear_regression_prediction = linear_regression_model.predict(input_data)[0]
svr_prediction = svr_model.predict(input_data)[0]
decision_tree_prediction = decision_tree_model.predict(input_data)[0]

In [ ]:
linear_regression_mse = mean_squared_error(y_test, linear_regression_predictions)
svr_mse = mean_squared_error(y_test, svr_predictions)
decision_tree_mse = mean_squared_error(y_test, decision_tree_predictions)

In [ ]:
best_model = None

In [ ]:
if linear_regression_mse <= svr_mse and linear_regression_mse <= decision_tree_mse:
    best_model = 'Linear Regression'
    best_prediction = linear_regression_prediction
elif svr_mse <= linear_regression_mse and svr_mse <= decision_tree_mse:
    best_model = 'Support Vector Regression'
    best_prediction = svr_prediction
else:
    best_model = 'Decision Tree'
    best_prediction = decision_tree_prediction

In [ ]:
input_data = {
    'name': 'fahad',
    'email': 'fahad@outlook.com'
}

In [ ]:
if input_data:
        name = input_data.get('name')
        email = input_data.get('email')

In [ ]:
c.execute('''INSERT INTO predictions (age, sex, bmi, children, smoker, region,
              linear_regression_prediction, svr_prediction, decision_tree_prediction, best_model, email, name)
              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
              (age, sex, bmi, children, smoker, region,
              linear_regression_prediction, svr_prediction, decision_tree_prediction, best_model, email, name))

In [ ]:
c.close()
conn.close()

In [ ]:
print(best_prediction)

In [ ]:
def jsonify_best_prediction(best_prediction):
    return jsonify(best_prediction)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
